# The Battle of Neighborhoods (Week 2) - Simone's assignment

#### Data download

In [1]:
!wget -q -O 'Stations_data.csv' http://web.mta.info/developers/data/nyct/subway/Stations.csv
print('Data downloaded!')

Data downloaded!


In [2]:
!wget -q -O 'Number_of_transits.xlsx' https://new.mta.info/document/16136
print('Data downloaded!')

Data downloaded!


#### Data import and cleanup

In [3]:
import pandas as pd
transit_weekday = pd.read_excel(r'Number_of_transits.xlsx', 'Average Weekday', header=1, skipfooter=9)
transit_weekend = pd.read_excel(r'Number_of_transits.xlsx', 'Average Weekend', header=1, skipfooter=9)
transit_annual = pd.read_excel(r'Number_of_transits.xlsx', 'Annual Total', header=1, skipfooter=9)

In [4]:
transit_weekday.drop(['Unnamed: 1', '2018-2019 Change', 'Unnamed: 10', '2019 Rank'], axis=1, inplace=True)
transit_weekday.columns= transit_weekday.columns.astype(str)
transit_weekday = transit_weekday.dropna()
transit_weekday = transit_weekday.rename(columns={'Station (alphabetical by borough)': 'Station', 'Boro': 'Borough'})
transit_weekday['Station'] = transit_weekday['Station'].apply(lambda x: x.split(' (')[0])
transit_weekday = transit_weekday.groupby('Station').mean().reset_index()
transit_weekday.reset_index()

transit_weekend.drop(['Unnamed: 1', '2018-2019 Change', 'Unnamed: 10', '2019 Rank'], axis=1, inplace=True)
transit_weekend.columns= transit_weekend.columns.astype(str)
transit_weekend = transit_weekend.dropna()
transit_weekend = transit_weekend.rename(columns={'Station (alphabetical by borough)': 'Station', 'Boro': 'Borough'})
transit_weekend['Station'] = transit_weekend['Station'].apply(lambda x: x.split(' (')[0])
transit_weekend = transit_weekend.groupby('Station').mean().reset_index()
transit_weekend.reset_index()

transit_annual.drop(['Unnamed: 1', '2018-2019 Change', 'Unnamed: 10', '2019 Rank'], axis=1, inplace=True)
transit_annual.columns= transit_annual.columns.astype(str)
transit_annual = transit_annual.dropna()
transit_annual = transit_annual.rename(columns={'Station (alphabetical by borough)': 'Station', 'Boro': 'Borough'})
transit_annual['Station'] = transit_annual['Station'].apply(lambda x: x.split(' (')[0])
transit_annual = transit_annual.groupby('Station').mean().reset_index()
transit_annual.reset_index()

,index,Station,2014,2015,2016,2017,2018,2019
0,0,1 Av,8.111070e+06,7.702110e+06,7.660896e+06,7071729.0,6.743140e+06,5.345371e+06
1,1,103 St,3.651225e+06,3.270322e+06,3.369575e+06,3329066.0,3.256210e+06,3.101329e+06
2,2,103 St-Corona Plaza,6.464627e+06,6.527196e+06,6.504644e+06,6498237.0,6.319564e+06,6.399657e+06
3,3,104 St,6.080170e+05,6.899725e+05,7.387250e+05,531425.0,5.028180e+05,6.559060e+05
4,4,110 St,3.955522e+06,4.209449e+06,4.196711e+06,3734660.0,3.394176e+06,3.316061e+06
...,...,...,...,...,...,...,...,...
334,334,Woodhaven Blvd,4.315562e+06,4.335814e+06,4.329110e+06,4192042.5,3.940138e+06,3.859460e+06
335,335,Woodlawn,2.375663e+06,2.418890e+06,2.380900e+06,2242100.0,2.160859e+06,2.094285e+06
336,336,Woodside-61 St,5.356621e+06,5.432682e+06,5.511564e+06,5457171.0,5.282486e+06,5.345369e+06
337,337,York St,2.760385e+06,2.900416e+06,3.286640e+06,3172717.0,3.256154e+06,3.927129e+06


In [5]:
stations_data = pd.read_csv ('Stations_data.csv')
stations_data.drop(['Station ID', 'Complex ID', 'GTFS Stop ID', 'Division', 'Structure', 'North Direction Label', 'South Direction Label', 'ADA', 'ADA Notes'], axis=1, inplace=True)
stations_data = stations_data.rename(columns={'Stop Name': 'Station'})
stations_data = stations_data.dropna()
stations_data = stations_data.groupby('Station').mean().reset_index()
stations_data

,Station,GTFS Latitude,GTFS Longitude
0,1 Av,40.730953,-73.981628
1,103 St,40.795379,-73.959104
2,103 St-Corona Plaza,40.749865,-73.862700
3,104 St,40.688445,-73.841006
4,110 St,40.795020,-73.944250
...,...,...,...
374,Woodlawn,40.886037,-73.878751
375,Woodside-61 St,40.745630,-73.902984
376,World Trade Center,40.712582,-74.009781
377,York St,40.701397,-73.986751


#### Import transit data into stations data

In [6]:
stations_data = pd.merge(stations_data, transit_weekday, on='Station')
stations_data.drop(['2014', '2015', '2016', '2017', '2018'], axis=1, inplace=True)
stations_data = stations_data.rename(columns={'2019': 'average_weekday'})

stations_data = pd.merge(stations_data, transit_weekend, on='Station')
stations_data.drop(['2014', '2015', '2016', '2017', '2018'], axis=1, inplace=True)
stations_data = stations_data.rename(columns={'2019': 'average_weekend'})

stations_data = pd.merge(stations_data, transit_annual, on='Station')
stations_data.drop(['2014', '2015', '2016', '2017', '2018'], axis=1, inplace=True)
stations_data = stations_data.rename(columns={'2019': 'total_annual'})

stations_data.head()

,Station,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual
0,1 Av,40.730953,-73.981628,18393.11020,12273.23080,5.345371e+06
1,103 St,40.795379,-73.959104,9982.05510,10230.49360,3.101329e+06
2,103 St-Corona Plaza,40.749865,-73.862700,19943.12990,24170.55770,6.399657e+06
3,104 St,40.688445,-73.841006,2275.30115,1408.16345,6.559060e+05
4,110 St,40.795020,-73.944250,10579.86610,11426.80770,3.316061e+06


In [7]:
stations_data.describe()

,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual
count,312.000000,312.000000,312.000000,312.000000,3.120000e+02
mean,40.729926,-73.926670,11927.684428,11947.777434,3.687655e+06
std,0.081160,0.057053,19228.375838,19715.711250,5.941098e+06
min,40.576312,-74.030876,243.366200,191.827000,8.843900e+04
25%,40.677248,-73.974331,4291.132850,4126.875025,1.336735e+06
50%,40.713387,-73.933663,6830.195900,6654.500050,2.097380e+06
75%,40.790421,-73.890532,12514.951800,13155.572125,3.887870e+06
max,40.903125,-73.755405,205159.090500,234114.326900,6.502029e+07


## Visualize stations on map

#### Use geopy library to get the latitude and longitude values of Toronto.

In [9]:
#!conda install -c conda-forge geopy --yes # only for the first run
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-2.1.0          | 64 KB     

#### Get geographical coordinates of New York

In [10]:
address = 'New York, US'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York are 40.7127281, -74.0060152.


#### Create the map centered on New York and superimpose the Stations

In [11]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

maximum = max(stations_data['total_annual'])

# add markers to map
for lat, lng, label, size in zip(stations_data['GTFS Latitude'], stations_data['GTFS Longitude'], stations_data['Station'], stations_data['total_annual']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10*size/maximum,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Getting data on competitors via Foursquare API

#### Foursquare credentials

In [12]:
CLIENT_ID = 'VTHYOWUVC51D3W3W5IPZRHMFTZOWC43X5I5AMNDJRGFSTBXR' # your Foursquare ID
CLIENT_SECRET = 'PLE5BUDZ0XQEPYYWWGFCQX30KARGTBXTCUOYDFOO055J33QO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 1000 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VTHYOWUVC51D3W3W5IPZRHMFTZOWC43X5I5AMNDJRGFSTBXR
CLIENT_SECRET:PLE5BUDZ0XQEPYYWWGFCQX30KARGTBXTCUOYDFOO055J33QO


#### Foursquare API to get venues

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### code to run the above function on each neighborhood and create a new dataframe called _newyork_venues_

In [14]:
station_latitude = stations_data.loc[:, 'GTFS Latitude']
station_longitude = stations_data.loc[:, 'GTFS Longitude']
station_name = stations_data.loc[:, 'Station']

#### Get venues from Foursquare

In [15]:
import requests # library to handle requests

In [16]:
# newyork_venues = getNearbyVenues(station_name, station_latitude, station_longitude, radius=100)

#### Save New York venues in a file (to use them when reaching the download limit)

In [17]:
#Import data from file when Foursquare reached the limit
newyork_venues = pd.read_csv('venues.csv')
#newyork_venues.to_csv(r'venues.csv', index=False, header=True)

In [18]:
print(newyork_venues.shape)
newyork_venues.head()

(2137, 7)


,Station,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1 Av,40.730953,-73.981628,Hawa Smoothies & Bubble Tea,40.730950,-73.981545,Juice Bar
1,1 Av,40.730953,-73.981628,Trader Joe's,40.730828,-73.980955,Grocery Store
2,1 Av,40.730953,-73.981628,Veeray Da Dhaba,40.730784,-73.982716,Indian Restaurant
3,1 Av,40.730953,-73.981628,Lower East Side Coffee Shop,40.730468,-73.980657,Diner
4,1 Av,40.730953,-73.981628,Domino's Pizza,40.730343,-73.980757,Pizza Place


#### Grouping of possible venues categories

In [19]:
list = newyork_venues.groupby("Venue Category").count()
#list.to_csv(r'categories.csv', index=True, header=True)
list

,Station,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,1,1,1,1,1,1
Accessories Store,3,3,3,3,3,3
Adult Boutique,1,1,1,1,1,1
African Restaurant,2,2,2,2,2,2
American Restaurant,25,25,25,25,25,25
...,...,...,...,...,...,...
Wine Bar,6,6,6,6,6,6
Wine Shop,17,17,17,17,17,17
Wings Joint,4,4,4,4,4,4


#### Load the file with the one hot encoding for place of interests

In [20]:
food_onecoding = pd.read_excel(r'coding.xlsx', header=0)
food_onecoding

,Venue Category,Snack place,Restaurant
0,ATM,0,0
1,Accessories Store,0,0
2,Adult Boutique,0,0
3,African Restaurant,0,1
4,American Restaurant,0,1
...,...,...,...
269,Wine Bar,0,0
270,Wine Shop,0,0
271,Wings Joint,0,0
272,Women's Store,0,0


In [21]:
newyork_venues = pd.merge(newyork_venues, food_onecoding, on='Venue Category')
newyork_venues_categorized = newyork_venues.groupby('Station').sum()
newyork_venues_categorized.drop(['Station Latitude', 'Station Longitude', 'Venue Latitude', 'Venue Longitude'], axis=1, inplace=True)
newyork_venues_categorized

,Snack place,Restaurant
Station,,
1 Av,1,3
103 St,0,0
103 St-Corona Plaza,5,3
104 St,0,0
110 St,4,1
...,...,...
Woodhaven Blvd,1,0
Woodlawn,2,0
Woodside-61 St,4,5


## Merging the data: traffic at stations and number of nearby snack places

In [22]:
stations_venues = pd.merge(stations_data, newyork_venues_categorized, on='Station')
stations_venues

,Station,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
0,1 Av,40.730953,-73.981628,18393.11020,12273.23080,5.345371e+06,1,3
1,103 St,40.795379,-73.959104,9982.05510,10230.49360,3.101329e+06,0,0
2,103 St-Corona Plaza,40.749865,-73.862700,19943.12990,24170.55770,6.399657e+06,5,3
3,104 St,40.688445,-73.841006,2275.30115,1408.16345,6.559060e+05,0,0
4,110 St,40.795020,-73.944250,10579.86610,11426.80770,3.316061e+06,4,1
...,...,...,...,...,...,...,...,...
293,Woodhaven Blvd,40.713493,-73.860402,12473.82680,12498.74040,3.859460e+06,1,0
294,Woodlawn,40.886037,-73.878751,6679.57870,7255.34610,2.094285e+06,2,0
295,Woodside-61 St,40.745630,-73.902984,16683.89370,20097.03850,5.345369e+06,4,5
296,York St,40.701397,-73.986751,12638.32680,13023.86540,3.927129e+06,0,0


In [39]:
stations_venues.describe()

,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
count,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000
mean,0.892617,40.728969,-73.927351,0.058079,0.051137,0.056454,0.181208,0.172260
std,1.234416,0.080972,0.056495,0.095782,0.086046,0.093398,0.176715,0.207093
min,0.000000,40.576312,-74.030876,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,40.675781,-73.974493,0.019745,0.016651,0.019175,0.000000,0.000000
50%,0.000000,40.713604,-73.933663,0.032620,0.027627,0.031228,0.100000,0.111111
75%,1.000000,40.779834,-73.892165,0.060720,0.056660,0.059174,0.300000,0.222222
max,3.000000,40.903125,-73.755405,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
from sklearn.preprocessing import MinMaxScaler
df = stations_venues[['average_weekday', 'average_weekend', 'total_annual','Snack place', 'Restaurant']]
# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
stations_venues[['average_weekday', 'average_weekend', 'total_annual','Snack place', 'Restaurant']] = df_norm
stations_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Station,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
0,1 Av,40.730953,-73.981628,0.088572,0.051647,0.080961,0.1,0.333333
1,103 St,40.795379,-73.959104,0.047525,0.042914,0.046401,0.0,0.000000
2,103 St-Corona Plaza,40.749865,-73.862700,0.096136,0.102507,0.097198,0.5,0.333333
3,104 St,40.688445,-73.841006,0.009916,0.005200,0.008739,0.0,0.000000
4,110 St,40.795020,-73.944250,0.050443,0.048029,0.049708,0.4,0.111111
...,...,...,...,...,...,...,...,...
293,Woodhaven Blvd,40.713493,-73.860402,0.059685,0.052611,0.058077,0.1,0.000000
294,Woodlawn,40.886037,-73.878751,0.031409,0.030196,0.030892,0.2,0.000000
295,Woodside-61 St,40.745630,-73.902984,0.080231,0.085093,0.080961,0.4,0.555556
296,York St,40.701397,-73.986751,0.060488,0.054856,0.059119,0.0,0.000000


## Cluster Stations

#### Run _k_-means to cluster the stations into 5 clusters.

In [24]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [25]:
# set number of clusters
kclusters = 4

stations_venues_clustering = stations_venues.drop(['Station', 'GTFS Latitude', 'GTFS Longitude'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(stations_venues_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 1, 0, 1, 0, 1, 0, 0, 1], dtype=int32)

#### Let's create a new dataframe that includes the cluster information

In [26]:
# add clustering labels
stations_venues.insert(1, 'Cluster Labels', kmeans.labels_)
stations_venues

,Station,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
0,1 Av,3,40.730953,-73.981628,0.088572,0.051647,0.080961,0.1,0.333333
1,103 St,0,40.795379,-73.959104,0.047525,0.042914,0.046401,0.0,0.000000
2,103 St-Corona Plaza,1,40.749865,-73.862700,0.096136,0.102507,0.097198,0.5,0.333333
3,104 St,0,40.688445,-73.841006,0.009916,0.005200,0.008739,0.0,0.000000
4,110 St,1,40.795020,-73.944250,0.050443,0.048029,0.049708,0.4,0.111111
...,...,...,...,...,...,...,...,...,...
293,Woodhaven Blvd,0,40.713493,-73.860402,0.059685,0.052611,0.058077,0.1,0.000000
294,Woodlawn,0,40.886037,-73.878751,0.031409,0.030196,0.030892,0.2,0.000000
295,Woodside-61 St,3,40.745630,-73.902984,0.080231,0.085093,0.080961,0.4,0.555556
296,York St,0,40.701397,-73.986751,0.060488,0.054856,0.059119,0.0,0.000000


#### Visualize the resulting clusters

In [27]:
import numpy as np # library to handle data in a vectorized manner

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stations_venues['GTFS Latitude'], stations_venues['GTFS Longitude'], stations_venues['Station'], stations_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

k clustering of snack, coffee and fast foods based on their location -> it is expected to indicate per each subway station the competition and the possibility of revenues

### Cluster 1

In [29]:
stations_venues.loc[stations_venues['Cluster Labels'] == 0]

,Station,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
1,103 St,0,40.795379,-73.959104,0.047525,0.042914,0.046401,0.0,0.000000
3,104 St,0,40.688445,-73.841006,0.009916,0.005200,0.008739,0.0,0.000000
5,116 St,0,40.801937,-73.948708,0.051248,0.051547,0.051033,0.2,0.000000
7,121 St,0,40.700492,-73.828294,0.010238,0.005428,0.009041,0.0,0.000000
8,125 St,0,40.809645,-73.948451,0.093729,0.096928,0.094029,0.1,0.000000
...,...,...,...,...,...,...,...,...,...
292,Winthrop St,0,40.656652,-73.950200,0.030530,0.016908,0.027623,0.0,0.111111
293,Woodhaven Blvd,0,40.713493,-73.860402,0.059685,0.052611,0.058077,0.1,0.000000
294,Woodlawn,0,40.886037,-73.878751,0.031409,0.030196,0.030892,0.2,0.000000
296,York St,0,40.701397,-73.986751,0.060488,0.054856,0.059119,0.0,0.000000


In [30]:
stations_venues.loc[stations_venues['Cluster Labels'] == 1]

,Station,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
2,103 St-Corona Plaza,1,40.749865,-73.862700,0.096136,0.102507,0.097198,0.5,0.333333
4,110 St,1,40.795020,-73.944250,0.050443,0.048029,0.049708,0.4,0.111111
6,116 St-Columbia University,1,40.807722,-73.964110,0.070927,0.047625,0.066101,0.6,0.000000
9,135 St,1,40.816062,-73.944209,0.046285,0.040645,0.044938,0.3,0.111111
14,145 St,1,40.824135,-73.943759,0.056312,0.060884,0.056935,0.4,0.111111
15,149 St-Grand Concourse,1,40.818393,-73.927035,0.062057,0.048969,0.059192,0.4,0.111111
22,168 St,1,40.840719,-73.939561,0.096799,0.081454,0.093449,0.5,0.111111
28,18 Av,1,40.619460,-73.989707,0.020880,0.019437,0.020383,0.4,0.111111
30,181 St,1,40.850600,-73.935782,0.052135,0.052531,0.051989,0.3,0.222222
32,183 St,1,40.858407,-73.903879,0.026342,0.025940,0.026039,0.3,0.222222


In [31]:
stations_venues.loc[stations_venues['Cluster Labels'] == 2]

,Station,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
13,14 St-Union Sq,2,40.735204,-73.990260,0.495757,0.511641,0.497396,0.1,0.000000
55,34 St-Herald Sq,2,40.749643,-73.987887,0.611251,0.582739,0.605204,0.0,0.111111
56,34 St-Penn Station,2,40.751330,-73.992224,0.408323,0.347413,0.395970,0.1,0.111111
187,Grand Central-42 St,2,40.751992,-73.977359,0.766315,0.448391,0.703157,0.5,0.111111
281,Times Sq-42 St,2,40.755356,-73.987042,1.000000,1.000000,1.000000,0.2,0.444444


In [32]:
stations_venues.loc[stations_venues['Cluster Labels'] == 3]

,Station,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
0,1 Av,3,40.730953,-73.981628,0.088572,0.051647,0.080961,0.1,0.333333
12,14 St,3,40.738982,-73.999367,0.220204,0.232085,0.221759,0.5,0.555556
16,15 St-Prospect Park,3,40.660365,-73.979493,0.029672,0.027077,0.028893,0.3,0.333333
18,157 St,3,40.834041,-73.944890,0.057576,0.051459,0.056234,0.4,0.555556
23,169 St,3,40.710470,-73.793604,0.037874,0.032715,0.036603,0.2,0.333333
...,...,...,...,...,...,...,...,...,...
267,Seneca Av,3,40.702762,-73.907740,0.014480,0.019947,0.015284,0.1,0.333333
286,Van Siclen Av,3,40.672061,-73.890480,0.012423,0.010429,0.011798,0.1,0.333333
287,Vernon Blvd-Jackson Av,3,40.742626,-73.953581,0.072748,0.059172,0.069837,0.5,0.777778
289,Wakefield-241 St,3,40.903125,-73.850620,0.020958,0.018560,0.020272,0.1,0.333333


In [33]:
stations_venues.loc[stations_venues['Cluster Labels'] == 4]

,Station,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant


In [34]:
stations_venues.loc[stations_venues['Cluster Labels'] == 0].describe()

,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
count,175.0,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.0,40.720400,-73.923060,0.041377,0.036763,0.040218,0.074857,0.062857
std,0.0,0.083184,0.057151,0.042990,0.037095,0.041345,0.077659,0.083808
min,0.0,40.576312,-74.014065,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,40.669922,-73.968988,0.015210,0.012631,0.014745,0.000000,0.000000
50%,0.0,40.701944,-73.926403,0.024375,0.022653,0.024979,0.100000,0.000000
75%,0.0,40.785160,-73.891102,0.050700,0.051548,0.049918,0.100000,0.111111
max,0.0,40.888022,-73.755405,0.242872,0.225808,0.225676,0.200000,0.333333


In [35]:
stations_venues.loc[stations_venues['Cluster Labels'] == 1].describe()

,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
count,49.0,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,1.0,40.756451,-73.930530,0.052192,0.043099,0.050162,0.418367,0.124717
std,0.0,0.090067,0.050841,0.043747,0.032540,0.041103,0.136433,0.097901
min,1.0,40.611486,-74.011056,0.009263,0.006165,0.008454,0.300000,0.000000
25%,1.0,40.685951,-73.960803,0.027244,0.023545,0.026039,0.300000,0.000000
50%,1.0,40.750582,-73.935782,0.035537,0.030389,0.033628,0.400000,0.111111
75%,1.0,40.840719,-73.904834,0.062057,0.052531,0.059192,0.500000,0.222222
max,1.0,40.898379,-73.783817,0.283509,0.184092,0.263742,1.000000,0.333333


In [36]:
stations_venues.loc[stations_venues['Cluster Labels'] == 2].describe()

,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
count,5.0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,2.0,40.748705,-73.986954,0.656329,0.578037,0.640345,0.180000,0.155556
std,0.0,0.007827,0.005737,0.234270,0.251236,0.231671,0.192354,0.168508
min,2.0,40.735204,-73.992224,0.408323,0.347413,0.395970,0.000000,0.000000
25%,2.0,40.749643,-73.990260,0.495757,0.448391,0.497396,0.100000,0.111111
50%,2.0,40.751330,-73.987887,0.611251,0.511641,0.605204,0.100000,0.111111
75%,2.0,40.751992,-73.987042,0.766315,0.582739,0.703157,0.200000,0.111111
max,2.0,40.755356,-73.977359,1.000000,1.000000,1.000000,0.500000,0.444444


In [37]:
stations_venues.loc[stations_venues['Cluster Labels'] == 3].describe()

,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
count,69.0,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000
mean,3.0,40.729756,-73.931659,0.061268,0.055121,0.059790,0.282609,0.484702
std,0.0,0.066620,0.058489,0.060758,0.057868,0.059911,0.159923,0.171468
min,3.0,40.577621,-74.030876,0.006991,0.006456,0.006646,0.000000,0.333333
25%,3.0,40.692259,-73.981628,0.026752,0.024011,0.025588,0.200000,0.333333
50%,3.0,40.729846,-73.944890,0.044085,0.034417,0.041961,0.300000,0.444444
75%,3.0,40.747659,-73.883697,0.072748,0.062567,0.071849,0.400000,0.555556
max,3.0,40.903125,-73.793604,0.354856,0.350704,0.353482,0.700000,1.000000


In [38]:
stations_venues.loc[stations_venues['Cluster Labels'] == 4].describe()

,Cluster Labels,GTFS Latitude,GTFS Longitude,average_weekday,average_weekend,total_annual,Snack place,Restaurant
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Final decision needs to consider:
- time of commuting people
- flux sense (e.g. all people tend to walk in a specific direction, the analysis gets data on the radius of a station)
- Data of some stations are merged and there are multiple exits
- qualitative check of the area and existing snack places, but the analysis helps in restricting the search